In [3]:
import folium
from folium import plugins
import requests
import json
import numpy as np
from pyArango.connection import *
from scipy.spatial.distance import pdist
import os
from time import sleep
import ConfigParser
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

ImportError: No module named pyArango.connection

In [ ]:
colors = '#58355e #e03616 #fff689 #cfffb0 #5998c5 #B5FFE1 #93E5AB #65B891 #4E878C #00241B'.split()
colors_for_gradient = '#6EEB83 #E4FF1A #E8AA14 #FF5714'.split()
color_gradient = dict(zip(np.around(np.linspace(1,0,len(colors_for_gradient)),2),colors_for_gradient))

In [ ]:
conn = Connection(username='galvbnk', password='')
db   = conn['capstone']
col  = db['radar']

In [ ]:
config = ConfigParser.RawConfigParser()
config.read('./secrets.conf')

In [ ]:
search_types = [term.strip() for term in open('./placetypes.txt').readlines()]

# Helper Functions

In [ ]:
def check_cached(lat=0.0, lon=0.0, terms = {}, meters=500):
    aql = ( "for doc in near('radar', @latt, @lonn, 100, @meters) filter doc.keyword == @keyword && "
            "doc.name == @name && doc.type == @type return doc")
    bind_vars = {'latt': lat, 'lonn': lon, 'meters': str(meters), 'type': '', 'keyword': '', 'name': ''}
    bind_vars.update(terms)
    qres = db.AQLQuery(aql, rawResults=False, batchSize=100, bindVars=bind_vars)
    
    return qres.response['result']

In [ ]:
def radar(coords, termdict, db_col, force=False, radius=20000, key='', do_cache=True):
    basetermdict = {'type': '', 'keyword': '', 'name': ''}
    params = {'location': '{0},{1}'.format(*coords),
              'radius': radius,
              'key': key}
    basetermdict.update(termdict)
    params.update(basetermdict)
    
    root = 'https://maps.googleapis.com/maps/api/place/radarsearch/json'
    
    cached = check_cached(coords[0], coords[1], terms=basetermdict)
    collection = []
    
    if len(cached) > 0 and not force:
        print('Cache hit: {0} {1}'.format(basetermdict, coords))
        collection = parse_gresults(cached[0]['response']['results'], params)
    else:
        print('Cache miss: {0} {1}'.format(basetermdict, coords))
        print('requesting...')
        r = requests.get(root, params=params, timeout=3)
        if r.status_code == requests.codes.ok:
            result_doc = r.json()
            if result_doc['status'] != 'OK':
                print(result_doc.keys())
                print('[ERR]{0} request failed: {1}'.format(result_doc['status'],''))#result_doc['error_message']))
                return collection
            print('[OK] status complete')
            collection = parse_gresults(result_doc['results'], params)

            if do_cache:
                doc = db_col.createDocument(initValues=params)
                doc['location'] = coords
                doc['response'] = result_doc
                doc.save()
            print('cached: complete')

    return collection

In [ ]:
def parse_gresults(results, params):
    collection = {'type': 'FeatureCollection', 'features': [] }

    for v in results:
        collection['features'].append({'properties': {'name':params['keyword']}, 'type': 'Feature', 'geometry': { 'type': 'Point',
                       'coordinates': [float(v['geometry']['location']['lng']),float(v['geometry']['location']['lat'])]}})
    return collection

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return np.r_[rho, phi]

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
#     print(x.shape, y.shape)
    return np.vstack((x, y)).T

In [ ]:
def rayleigh_upsample(loc=np.array([0.0,0.0]), mean_shift=1, samples=20, mean=2):
    magnitude = np.random.rayleigh(mean, size=samples) * (mean_shift * 1.0 / mean)
    direction = np.random.rand(samples) * 2 * 3.141559
    return np.atleast_2d(loc) + pol2cart(magnitude, direction)

In [ ]:
def plot_parsed_res():
    

# Main Driver

In [ ]:
location = [39.7047, -105.0814]

searches = 'grocery;chipotle;'.split(';')

res = {}
for search in [x for x in searches if len(x) > 0]:
    kwparams = {}
    if search in search_types:
        kwparams['type'] = search
    else:
        kwparams['keyword'] = search
    res[search] = radar(location, kwparams, col, radius=25000, key=config.get('keys', 'google_places_api_key'))

In [ ]:
for k,v in res.iteritems():
    print("{0}: {1}".format(k, len(v['features'])))

# Marker Map

In [ ]:
wm = folium.Map(location=location, zoom_start=13)
for idx, kv in enumerate(res.iteritems()):
    if kv[0] not in ['light rail stations']:  continue
    print('plotting ' + kv[0])
    for feat in kv[1]['features']:
#         folium.CircleMarker(location=feat['geometry']['coordinates'][::-1], 
#                             radius=2000,
#                             fill_opacity=.075,
#                             fill_color='#B49FCC',
#                             color=None).add_to(wm)
        folium.CircleMarker(location=feat['geometry']['coordinates'][::-1], 
                            radius=100,
                            fill_opacity=1.0,
                            color= '#000000', #colors[idx],
                            fill_color=None).add_to(wm)
        upsamp = rayleigh_upsample(feat['geometry']['coordinates'][::-1], mean_shift=.01, samples=100, mean=2)
        for sub in range(upsamp.shape[0]):
            folium.CircleMarker(location=upsamp[sub,:], 
                                radius=75,
                                fill_opacity=.6,
                                color= colors[idx],
                                ).add_to(wm)
wm

In [ ]:
res['chipotle']['features'][0]['geometry']

# Heatmap

In [ ]:
wm = folium.Map(location=location, zoom_start=13)
dists = {}

for k, v in res.iteritems():
    if k not in ['grocery']: continue
    ps = [feat['geometry']['coordinates'][::-1] for feat in v['features']]
    dists[k] = { 'data':ps, 'dist':pdist(ps, 'euclidean') }

allp = np.array([inner for k, v in dists.iteritems() for inner in v['data'] ])

plt.figure(figsize=(13,13))

wm.add_children(plugins.HeatMap(data=allp,
                               blur=30,
                               radius=90,
                               gradient=color_gradient,
                               min_opacity=.27,
                               max_val=.6,
                               max_zoom=30))
wm

In [ ]:
plt.figure(figsize=(13,15))
c = 1
for k,v in dists.iteritems():
    plt.subplot(4,2,c)
    plt.hist(v['dist'], bins=25, label=str(k)+str(len(v['dist'])), alpha=.6);
    if k == 'grocery': plt.hist(np.random.rayleigh(1.7, 12720) * (0.16 / 2.0), bins=25, label='rayleigh ~( $\sigma$ 1.7 )', alpha=.37)
    plt.axvline(v['dist'].mean(), c='b', linestyle='-.', label='{0:.2}'.format(v['dist'].mean()))
    c += 1
    plt.legend()

In [ ]:
samp = rayleigh_upsample(samples=50000)
plt.figure(figsize=(10,10))
plt.hexbin(samp[:,0], samp[:,1], cmap=plt.cm.viridis, mincnt=1);